# Import

In [1]:
!pip install mysql-connector-python

In [1]:
import mysql.connector
from datetime import date

# Connection

In [2]:
cnx = mysql.connector.connect(host='172.26.0.2',port='3306',database='nico_studienarbeit',user='studienarbeit',password='dbstuar2020')

In [3]:
cursor = cnx.cursor()

# Querys

If string include ' ' to resole errors in the statement!

## Insert routines
1. Buch: Sorte, Verlag, Buch
2. Hörbuch: Sorte, Verlag, Buch, Person, Sprecher, Hörbuch
3. Ebook: Sorte, Verlag, Buch, Person, Ebook
4. Person: Person
5. Ausleiher: Person, Ausleiher
6. Sorte: Sorte
7. Autor: Person, Autor
8. Wort: Wort
9. Maler: Person, Maler

In [4]:
def insert_db(INSERT, values, SELECT=None):
    val = [v for v in values.values()]

    first_result = None
    if SELECT:
        cursor.execute(SELECT, val)
        first_result = cursor.fetchone()
            
    if not first_result:
        cursor.execute(INSERT, val)
        cnx.commit()
        return cursor.lastrowid

    return first_result[0]

def insert_db_ntm(INSERT, values, SELECT=None):
    val = [v for v in values.values()]

    first_result = None
    if SELECT:
        cursor.execute(SELECT, val)
        first_result = cursor.fetchone()
            
    if not first_result:
        val = ["nt" + str(hash(str(values)))[-11:]] + val
        cursor.execute(INSERT, val) 
        cnx.commit()
        return cursor.lastrowid

    return first_result[0]

### Buch
INSERT_SORTE, INSERT_VERLAG, INSERT_AUTOR

In [5]:
INSERT_SORTE = "INSERT INTO Sorte(Name,Beschreibung) VALUES(%s,%s)"
SELECT_SORTE = "SELECT SorteId FROM Sorte WHERE Name=%s AND Beschreibung=%s;"

In [6]:
INSERT_VERLAG = "INSERT INTO Verlag(Kurzname, Name, Postleitzahl, Strasse, Internetadresse, Beschreibung) VALUES(%s,%s,%s,%s,%s,%s)"
SELECT_VERLAG = "SELECT VerlagId FROM Verlag WHERE Kurzname=%s AND Name=%s AND Postleitzahl=%s AND Strasse=%s AND Internetadresse=%s AND Beschreibung=%s;"

In [39]:
INSERT_BUCH = "INSERT IGNORE INTO Buch(ISBN,Titel,Untertitel,VerlagId,Erscheinungsjahr,SorteId,Kurzbeschreibung,Preis,Auflage,Sprache) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"

In [8]:
INSERT_AUTORBUCHZUORD = "INSERT IGNORE INTO AutorBuchZuord(AutorId,ISBN) VALUES(%s,%s);"

In [9]:
def insert_book(val_schlagworte, val_verlag, val_buch, val_person, val_autor, val_sorte):
    try:
        print("--- Inserting Sorte")
        sorte_id = insert_db(INSERT_SORTE, val_sorte, SELECT_SORTE)
        val_buch["SorteId"] = sorte_id

        print("--- Inserting Verlag")
        verlag_id = insert_db(INSERT_VERLAG, val_verlag, SELECT_VERLAG)
        val_buch["VerlagId"] = verlag_id
        
        print("--- Inserting Buch")
        buch_id = insert_db(INSERT_BUCH, val_buch)

        print("--- Inserting Schlagworte")
        for schlagwort in val_schlagworte:
            print("--- --- Inserting Schlagwort")
            insert_db(INSERT_SCHLAGWORT, schlagwort)

            print("--- --- Inserting MediumBuchZuord")
            val_mediumwortzuord = {
                "MediumId": buch_id,
                "Wort": schlagwort["Wort"]
            }
            insert_db(INSERT_MEDIUMWORTZUORD, val_mediumwortzuord)

        print("--- Inserting Person")
        person_id = insert_db(INSERT_PERSON, val_person, SELECT_PERSON)
        val_autor["PersonenId"] = person_id
        
        print("--- Inserting Autor")
        autor_id = insert_db(INSERT_AUTOR, val_autor, SELECT_AUTOR)

        print("--- Inserting AutorBuchZuord")
        val_autorbuchzuord = {
            "AutorId" : autor_id,
            "IBSN" : val_buch["ISBN"]
        }
        insert_db(INSERT_AUTORBUCHZUORD, val_autorbuchzuord)
        print("--- Done")
        
    except Exception as e:
        print(e)
        return False
    
    return True

### Hörbuch
INSERT_SORTE, INSERT_VERLAG, INSERT_BUCH, INSERT_SPRECHER

In [10]:
INSERT_PERSON = "INSERT INTO Person(Vorname,Name,Email,Geburtsdatum) VALUES(%s,%s,%s,%s);"
SELECT_PERSON = "SELECT PersonenId FROM Person WHERE Vorname=%s AND Name=%s AND Email=%s AND Geburtsdatum=%s;"

In [11]:
INSERT_SPRECHER = "INSERT INTO Sprecher(PersonenId,Beschreibung) VALUES(%s,%s)"
SELECT_SPRECHER = "SELECT SprecherId FROM Sprecher WHERE PersonenId=%s AND Beschreibung=%s;"

In [12]:
INSERT_HOERBUCH = "INSERT IGNORE INTO Hoerbuch(ISBN,BuchISBN,SprecherId,VerlagId) VALUES(%s,%s,%s,%s);"

In [13]:
def insert_hoerbuch(val_person, val_sprecher, val_verlag, val_hoerbuch):
    try:
        print("--- Inserting Person")
        person_id = insert_db(INSERT_PERSON, val_person, SELECT_PERSON)
        val_sprecher["PersonenId"] = person_id

        print("--- Inserting Sprecher")
        sprecher_id = insert_db(INSERT_SPRECHER, val_sprecher, SELECT_SPRECHER)
        val_hoerbuch["SprecherId"] = sprecher_id

        print("--- Inserting Verlag")
        verlag_id = insert_db(INSERT_VERLAG, val_verlag, SELECT_VERLAG)
        val_hoerbuch["VerlagId"] = verlag_id

        print("--- Inserting Hoerbuch")
        insert_db(INSERT_HOERBUCH, val_hoerbuch)
    
    except Exception as e:
        print(e)
        return False
    
    return True

### Ebook
INSERT_SORTE, INSERT_VERLAG, INSERT_BUCH, INSERT_PERSON

In [14]:
INSERT_EBOOK = "INSERT IGNORE INTO Ebooks(ISBN,BuchISBN,Dateiformat) VALUES(%s,%s,%s)"

In [15]:
def insert_ebook(val_ebook):
    try:
        print("--- Inserting Ebook")
        insert_db(INSERT_EBOOK, val_ebook)
    except Exception as e:
        print(e)
        return False
    
    return True

### Ausleiher
INSERT_PERSON

In [16]:
INSERT_AUSLEIHER = "INSERT INTO Ausleiher(PersonenId,Strasse,Postleitzahl,Ort,Telefonnummer) VALUES(%s,%s,%s,%s,%s)"
SELECT_AUSLEIHER = "SELECT AusleiherId FROM Ausleiher WHERE PersonenId=%s AND Strasse=%s AND Postleitzahl=%s AND Ort=%s AND Telefonnummer=%s;"

In [17]:
def insert_ausleiher(val_person, val_ausleiher):
    try:
        print("--- Inserting Person")
        person_id = insert_db(INSERT_PERSON, val_person, SELECT_PERSON)
        val_ausleiher["PersonenId"] = person_id

        print("--- Inserting Ausleiher")
        insert_db(INSERT_AUSLEIHER, val_ausleiher, SELECT_AUSLEIHER)

        print("--- Done")
    except Exception as e:
        print(e)
        return False
    
    return True

### Autor
INSERT_PERSON

In [18]:
INSERT_AUTOR = "INSERT INTO Autor(PersonenId,Beschreibung) VALUES(%s,%s)"
SELECT_AUTOR = "SELECT AutorId FROM Autor WHERE PersonenId=%s AND Beschreibung=%s;"

### Schlagwort

In [19]:
INSERT_SCHLAGWORT = "INSERT IGNORE INTO Schlagwort(Wort,Beschreibung) VALUES(%s,%s)"

### MediumWortZuord

In [20]:
INSERT_MEDIUMWORTZUORD = "INSERT IGNORE INTO MediumWortZuord(MediumId,Wort) VALUES(%s,%s)"

### Maler
INSERT_PERSON

In [21]:
INSERT_MALER = "INSERT INTO Maler(PersonenId,Beschreibung) VALUES(%s,%s)"
SELECT_MALER = "SELECT MalerId FROM Maler WHERE PersonenId=%s AND Beschreibung=%s;"

### NichtTextMedien
INSERT_SORTE

In [22]:
INSERT_NICHTTEXTMEDIEN = "INSERT INTO NichtTextMedien(NichtTextMedienId,Titel,Untertitel,Erscheinungsjahr,Kurzbeschreibung,SorteId,Typ) VALUES(%s,%s,%s,%s,%s,%s,%s)"
SELECT_NICHTTEXTMEDIEN = "SELECT NichttextmedienId FROM NichtTextMedien WHERE Titel=%s AND Untertitel=%s AND Erscheinungsjahr=%s AND Kurzbeschreibung=%s AND SorteId=%s AND Typ=%s;"

### Bild
INSERT_SORTE, INSERT_NICHTTEXTMEDIEN, INSERT_PERSON, INSERT_MALER

In [23]:
INSERT_BILD = "INSERT INTO Bild(NichtTextMedienId,Bild,MalerId) VALUES(%s,%s,%s)"
SELECT_BILD = "SELECT BildId FROM Bild WHERE NichtTextMedienId=%s AND Bild=%s AND MalerId=%s;"

In [55]:
def insert_bild(val_sorte, val_person, val_maler, val_nichttextmedien, val_bild, val_schlagworte):
    try:
        print("--- Inserting Sorte")
        sorte_id = insert_db(INSERT_SORTE, val_sorte, SELECT_SORTE)
        val_nichttextmedien["SorteId"] = sorte_id

        print("--- Inserting Person")
        person_id = insert_db(INSERT_PERSON, val_person, SELECT_PERSON)
        val_maler["PersonenId"] = person_id
        
        print("--- Inserting Maler")
        maler_id = insert_db(INSERT_MALER, val_maler, SELECT_MALER)
        val_bild["MalerId"] = maler_id

        print("--- Inserting NichtTextMedien")
        val_nichttextmedien["Typ"] = "Bild"
        ntm_id = insert_db_ntm(INSERT_NICHTTEXTMEDIEN, val_nichttextmedien, SELECT_NICHTTEXTMEDIEN)
        val_bild["NichtTextMedienId"] = ntm_id
        
        print("--- Inserting Schlagworte")
        for schlagwort in val_schlagworte:
            print("--- --- Inserting Schlagwort")
            insert_db(INSERT_SCHLAGWORT, schlagwort)

            print("--- --- Inserting MediumBuchZuord")
            val_mediumwortzuord = {
                "MediumId": ntm_id,
                "Wort": schlagwort["Wort"]
            }
            insert_db(INSERT_MEDIUMWORTZUORD, val_mediumwortzuord)

        print("--- Inserting Bild")
        insert_db(INSERT_BILD, val_bild, SELECT_BILD)

        print("--- Done")        
    except Exception as e:
        print(e)
        return False
    
    return True

### Video
INSERT_SORTE, INSERT_NICHTTEXTMEDIEN

In [25]:
INSERT_VIDEO = "INSERT INTO Video(NichtTextMedienId,Sprache) VALUES(%s,%s)"
SELECT_VIDEO = "SELECT VideoId FROM Video WHERE NichtTextMedienId=%s AND Sprache=%s;"

In [58]:
def insert_video(val_sorte, val_nichttextmedien, val_video, val_schlagworte):
    try:
        print("--- Inserting Sorte")
        sorte_id = insert_db(INSERT_SORTE, val_sorte, SELECT_SORTE)
        val_nichttextmedien["SorteId"] = sorte_id

        print("--- Inserting NichtTextMedien")
        val_nichttextmedien["Typ"] = "Video"
        ntm_id = insert_db_ntm(INSERT_NICHTTEXTMEDIEN, val_nichttextmedien, SELECT_NICHTTEXTMEDIEN)
        val_video["NichtTextMedienId"] = ntm_id
        
        print("--- Inserting Schlagworte")
        for schlagwort in val_schlagworte:
            print("--- --- Inserting Schlagwort")
            insert_db(INSERT_SCHLAGWORT, schlagwort)

            print("--- --- Inserting MediumBuchZuord")
            val_mediumwortzuord = {
                "MediumId": ntm_id,
                "Wort": schlagwort["Wort"]
            }
            insert_db(INSERT_MEDIUMWORTZUORD, val_mediumwortzuord)

        print("--- Inserting Video")
        insert_db(INSERT_VIDEO, val_video, SELECT_VIDEO)

        print("--- Done")        
    except Exception as e:
        print(e)
        return False
    
    return True

### Ausleihe
INSERT_AUSLEIHER, INSERT_NICHTTEXTMEDIEN/INSERT_BUCH/INSERT_EBOOK/INSERT_HOERBUCH

In [51]:
INSERT_AUSLEIHE = "INSERT INTO Ausleihe(AusleiherId,MediumId) VALUES(%s,%s)"

In [52]:
def insert_ausleihe(val_person, val_ausleiher, ISBN):
    print("--- Inserting Person")
    person_id = insert_db(INSERT_PERSON, val_person, SELECT_PERSON)
    val_ausleiher["PersonenId"] = person_id

    print("--- Inserting Ausleiher")
    ausleiher_id = insert_db(INSERT_AUSLEIHER, val_ausleiher, SELECT_AUSLEIHER)

    val_ausleihe = {
        "AusleiherId": ausleiher_id,
        "MediumId": ISBN
    }
    insert_db(INSERT_AUSLEIHE, val_ausleihe)


# Testqueries

## [TEST] Buch

In [29]:
val_verlag = {
    "Kurzname" : "hi",
    "Name" : "hiverlag",
    "Postleitzahl" : "89078",
    "Strasse" : "Rubelstraße 33",
    "Internetadresse" : "hi@verlag.com",
    "Beschreibung" : "Für alles was high ist."
}

val_schlagwort = [
    {   "Wort": "Fun",
        "Beschreibung": "Fun ist eine tolle Kategorie."},
    {   "Wort": "Lust",
        "Beschreibung": "Lust im Herz"}
]

val_buch = {
    "ISBN": "1234567890123",
    "Titel": "WOW",
    "Untertitel": "Ein WOW für dich!",
    "VerlagId": "",
    "Erscheinungsjahr": "2017",
    "SorteId": "",
    "Kurzbeschreibung": "Klassebuch",
    "Preis": "9.99",
    "Auflage": "1",
    "Sprache": "deutsch"
}

val_person = {
    "Vorname": "Guenter",
    "Name": "Pauli",
    "Email": "guenter@pauli.sucks",
    "Geburtsdatum": date(2000, 12, 2)
}

val_autor = {
    "PersonenId": "",
    "Beschreibung": "Pauli aus Pauli"
}

val_sorte = {
    "Name": "Spassbuch",
    "Beschreibung": "Diese Bücher machen Spaß"
}

insert_book(val_schlagwort, val_verlag, val_buch, val_person, val_autor, val_sorte)


--- Inserting Sorte
--- Inserting Verlag
--- Inserting Buch
--- Inserting Schlagworte
--- --- Inserting Schlagwort
--- --- Inserting MediumBuchZuord
--- --- Inserting Schlagwort
--- --- Inserting MediumBuchZuord
--- Inserting Person
--- Inserting Autor
--- Inserting AutorBuchZuord
--- Done


True

## [TEST] Hoerbuch

In [30]:
val_verlag = {
    "Kurzname" : "hi_hoerbuch",
    "Name" : "hiverlag für Hörbücher",
    "Postleitzahl" : "89078",
    "Strasse" : "Rubelstraße 33",
    "Internetadresse" : "hihoeren@verlag.com",
    "Beschreibung" : "Für alles was high ist."
}

val_person = {
    "Vorname": "Manfred",
    "Name": "Manni",
    "Email": "manni@manii.sk",
    "Geburtsdatum": date(1970, 4, 1)
}

val_sprecher = {
    "PersonenId": "",
    "Beschreibung": "Spricht männliche charaktere"
}

val_hoerbuch = {
    "ISBN": "5678456456234",
    "BuchISBN": "1234567890123",
    "SprecherId": "",
    "VerlagId": ""
}

insert_hoerbuch(val_person, val_sprecher, val_verlag, val_hoerbuch)

--- Inserting Person
--- Inserting Sprecher
--- Inserting Verlag
--- Inserting Hoerbuch


True

## [TEST] Ebook

In [31]:

val_ebook = {
    "ISBN": "5678456498776",
    "BuchISBN": "1234567890123",
    "Dateiformat": "epub"
}

insert_ebook(val_ebook)

--- Inserting Ebook


True

## [TEST] Bild

In [60]:
val_sorte = {
    "Name": "Spaßbild",
    "Beschreibung": "Dieses Bild erzeugt Spaß"
}

val_person = {
    "Vorname": "Mali",
    "Name": "Maler",
    "Email": "mal@er.sk",
    "Geburtsdatum": date(1980, 2, 28)
}

val_maler = {
    "PersonenId": "",
    "Beschreibung": "Malt immer in Grau"
}

val_nichttextmedien = {
    "Titel": "WOW",
    "Untertitel": "Ein WOW für dich!",
    "Erscheinungsjahr": "2017",
    "Kurzbeschreibung": "Ein Bild von Klasse",
    "SorteId": "",
    "Typ": ""
}

with open("testimg_base64") as f:
    test_bild = f.read()
val_bild = {
    "NichtTextMedienId": "",
    "Bild": test_bild,
    "MalerId": ""
}

val_schlagworte=[]

insert_bild(val_sorte, val_person, val_maler, val_nichttextmedien, val_bild, val_schlagworte)

--- Inserting Sorte
--- Inserting Person
--- Inserting Maler
--- Inserting NichtTextMedien
--- Inserting Schlagworte
--- Inserting Bild
--- Done


True

# TODO Add Schlagworte  zu den anderen Sachen


## [TEST] Video

In [59]:
val_sorte = {
    "Name": "Spaßbild",
    "Beschreibung": "Dieses Bild erzeugt Spaß"
}

val_nichttextmedien = {
    "Titel": "18 Movie",
    "Untertitel": "18 klassiker",
    "Erscheinungsjahr": "1982",
    "Kurzbeschreibung": "Ein Video von Klasse",
    "SorteId": "",
    "Typ": ""
}

val_video = {
    "NichtTextMedienId": "",
    "Sprache": "deutsch"
}

val_schlagworte=[]

insert_video(val_sorte, val_nichttextmedien, val_video, val_schlagworte)

--- Inserting Sorte
--- Inserting NichtTextMedien
--- Inserting Schlagworte
--- Inserting Video
--- Done


True

## [TEST] Ausleiher

In [41]:
val_person = {
    "Vorname": "Max",
    "Name": "Mustermann",
    "Email": "max.mustermann@denken.de",
    "Geburtsdatum": date(1999, 6, 30)
}

val_ausleiher = {
    "PersonenId": "",
    "Strasse": "Nostreet 24",
    "Postleitzahl": "56789",
    "Ort": "Entenhausen",
    "Telefonnummer" : "23458/856765567"
}

insert_ausleiher(val_person, val_ausleiher)

--- Inserting Person
--- Inserting Ausleiher
--- Done


True

## [TEST] Ausleihe

In [54]:
val_person = {
    "Vorname": "Max",
    "Name": "Mustermann",
    "Email": "max.mustermann@denken.de",
    "Geburtsdatum": date(1999, 6, 30)
}

val_ausleiher = {
    "PersonenId": "",
    "Strasse": "Nostreet 24",
    "Postleitzahl": "56789",
    "Ort": "Entenhausen",
    "Telefonnummer" : "23458/856765567"
}

ISBN = "5678456498776"

insert_ausleihe(val_person, val_ausleiher, ISBN)

--- Inserting Person
--- Inserting Ausleiher


IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`nico_studienarbeit`.`Ausleihe`, CONSTRAINT `Ausleihe_ibfk_2` FOREIGN KEY (`MediumId`) REFERENCES `Buch` (`ISBN`))

# Execute

# Close

In [47]:
cursor.close()
cnx.close()